In [1]:
import fun as f
import math
import numpy as np
import time
import pickle
import os
import copy
import matplotlib.pyplot as plt
from matplotlib import pyplot
np.random.seed(400)

In [2]:
path = '/Users/user/Desktop/MSM/09-Deep Learning - KTH/assignments/assignment-2/Datasets/cifar-10-batches-py'
result_pics = '/Users/user/Desktop/MSM/09-Deep Learning - KTH/assignments/assignment-1/Result_Pics'

In [3]:
#read in data:
trainning_data = f.LoadBatch(os.path.join(path,'data_batch_1'))
validation_data = f.LoadBatch(os.path.join(path,'data_batch_2'))
test_data = f.LoadBatch(os.path.join(path,'test_batch'))

In [4]:
#initialize weights and bias:
W_1, b_1, W_2, b_2 = f.init_two_layers_w_b_param(trainning_data.data, trainning_data.labels, 50)

In [5]:
#Compute gradients analitically:
grad_b_1, grad_b_2, grad_W_1, grad_W_2 = f.ComputeGradsAnalt(
    trainning_data.data[:, 0:20], trainning_data.labels[:,0:20], b_1, b_2, W_1, W_2, lambda_=0)
#Compute gradients numerically:
#1.finite difference method.
grad_b_1_n, grad_b_2_n, grad_W_1_n, grad_W_2_n = f.ComputeGradsNum(
    trainning_data.data[:, 0:20], trainning_data.labels[:,0:20], W_1, W_2, b_1, b_2, h_=1e-6, lambda_=0)
#2.centered difference method.
grad_b_1_n_s, grad_b_2_n_s, grad_W_1_n_s, grad_W_2_n_s = f.ComputeGradsNumSlow(
    trainning_data.data[:, 0:20], trainning_data.labels[:,0:20], W_1, W_2, b_1, b_2, h_=1e-6, lambda_=0)

In [6]:
print('Max Relative error between gradients computed analitically and numerically using: finite diff method')
print(f.MaxRelativeError(grad_b_1, grad_b_1_n))
print(f.MaxRelativeError(grad_b_2, grad_b_2_n))
print(f.MaxRelativeError(grad_W_1, grad_W_1_n))
print(f.MaxRelativeError(grad_W_2, grad_W_2_n))
print('Max Relative error between gradients computed analitically and numerically using: centered diff method')
print(f.MaxRelativeError(grad_b_1, grad_b_1_n_s))
print(f.MaxRelativeError(grad_b_2, grad_b_2_n_s))
print(f.MaxRelativeError(grad_W_1, grad_W_1_n_s))
print(f.MaxRelativeError(grad_W_2, grad_W_2_n_s))

Max Relative error between gradients computed analitically and numerically using: finite diff method
8.844392605193683e-06
0.00010112146279395916
0.003626248845054414
6.050587200705587e-05
Max Relative error between gradients computed analitically and numerically using: centered diff method
8.546697380276077e-07
1.1557400414400866e-07
0.001961686602638457
8.903622176761293e-06


In [32]:
def ComputeAccuracy(X_, y_, W_1_, W_2_, b_1_, b_2_):
    """
    :param X_:  N x d matrix,  trainning images.
    :param y_:  N x 1 vector containing the trainning set true class.
    :param W_: K x d matrix. Weights.
    :param b_:  K x 1 vector. Bias.
    :return acc: scalar.
    """
    Npts = X_.shape[1]
    ind_size_n = np.ones((Npts,1))
    #S_1 = mxN matrix.
    S_1 = np.matmul(W_1_, X_) + np.matmul(b_1_, ind_size_n.T)
    #H = mxN matrix.
    H = f.ReLU(S_1)
    #S_2 = kxN matrix.
    S_2 = np.matmul(W_2_, H) + np.matmul(b_2_, ind_size_n.T)
    #P = kxN matrix.
    P = f.SOFTMAX(S_2)
    P_ = np.argmax(P,axis=0)
    acc = np.count_nonzero(y_== P_) / float(len(P_))
    
    return acc

In [5]:
cf_train = []
cf_val = []
cl_train = []
cl_val = []
acc_train = []
acc_val = []

GD_params = f.params(n_batch=10, eta=0.01, n_epochs=200)
W_1, b_1, W_2, b_2 = f.init_two_layers_w_b_param(
                                trainning_data.data[:, 0:100], trainning_data.labels[:,0:100], m_=50)

for epoch in range(200):
    for j in range(10):
        j_start = j * GD_params.n_batch
        j_end = (j + 1) * GD_params.n_batch
        Xbatch = trainning_data.data[:, 0:100][:, j_start:j_end]
        Ybatch = trainning_data.labels[:,0:100][:, j_start:j_end]

#         Xbatch = trainning_data.data[:, 0:100]
#         Ybatch = trainning_data.labels[:,0:100]

        grad_b_1_t, grad_b_2_t, grad_W_1_t, grad_W_2_t = f.ComputeGradsAnalt(
                                                    Xbatch, Ybatch, b_1, b_2, W_1, W_2, lambda_=0)

        W_1 = W_1 - GD_params.eta * grad_W_1_t
        b_1 = b_1 - GD_params.eta * grad_b_1_t
        W_2 = W_2 - GD_params.eta * grad_W_2_t
        b_2 = b_2 - GD_params.eta * grad_b_2_t

    # cost per epoch

    trainning_cost = f.ComputeCost(trainning_data.data, trainning_data.labels, W_1, W_2, b_1, b_2, lambda_=0)
    validation_cost =f.ComputeCost(validation_data.data, validation_data.labels, W_1, W_2, b_1, b_2, lambda_=0)

    # # loss per epoch
    # trainning_loss = f.ComputeCost(X_, Y_.transpose(), W_, b_, 0)
    # validation_loss = f.ComputeCost(X_val, Y_val.transpose(), W_, b_, 0)

    cf_train.append(trainning_cost)
    cf_val.append(validation_cost)

    # cl_train.append(trainning_loss)
    # cl_val.append(validation_loss)

    # accuracy per epoch
    acc_train_ = f.ComputeAccuracy(trainning_data.data, trainning_data.y, W_1, W_2, b_1, b_2)
    acc_val_ = f.ComputeAccuracy(validation_data.data, validation_data.y, W_1, W_2, b_1, b_2)

    acc_train.append(acc_train_)
    acc_val.append(acc_val_)

    
    
    
    

In [9]:
p, H = SOFTMAX(trainning_data.data, W_1, W_2, b_1, b_2)

In [24]:
p

array([[1.14036047e-02, 9.57492997e-03, 2.28200442e-04, ...,
        2.32133844e-02, 1.01212594e-01, 9.51380435e-04],
       [8.26587389e-03, 5.29646848e-03, 4.11120975e-04, ...,
        1.69641158e-02, 1.54951397e-02, 3.98445840e-03],
       [1.58021107e-03, 5.04170308e-03, 1.76807511e-04, ...,
        2.89415439e-03, 6.71610121e-02, 1.08673847e-02],
       ...,
       [1.68895008e-04, 1.67246031e-04, 3.59973789e-04, ...,
        7.65693320e-02, 9.30518032e-03, 7.70135595e-02],
       [4.30344914e-05, 8.00113703e-04, 5.16034872e-03, ...,
        1.79422939e-02, 3.48231603e-02, 9.86665920e-04],
       [1.43827406e-04, 9.76424857e-01, 9.92172233e-01, ...,
        5.13400625e-02, 6.30694424e-02, 5.75626014e-04]])

In [26]:
trainning_data.labels.shape

(10, 10000)

In [25]:
p.shape

(10, 10000)

In [19]:
p.T trainning_data.labels

(10000, 10000)

In [28]:
np.multiply(trainning_data.labels, p).shape

(10, 10000)

In [22]:
(np.sum(np.multiply(trainning_data.labels, p), axis=0)).shape

(10000,)

In [8]:
def SOFTMAX(X_, W_1_, W_2_, b_1_, b_2_):
    """ 
    :return p: kxN softmax
    """
    ind_size_n = np.ones((X_.shape[1],1))
    H_ = f.ReLU(X_, W_1_, b_1_)
    S_2_ = np.matmul(W_2_, H_) + np.matmul(b_2_, ind_size_n.T)
    p_ =  np.exp(S_2_) / np.matmul(np.ones((1, S_2_.shape[0])), np.exp(S_2_))
    
    #same as dividing by np.exp(s_)/ (np.sum(np.exp(s_), axis=0))
    return p_ , H_

def cross_entropy_loss(Y_, p_):
    """
    :param Y_:
    :param p_:
    :return l_cross_:
    """
    l_cross_ = -np.log(np.sum((np.multiply(Y_, p_)),axis=0))
    return l_cross_


In [6]:
acc_train

[0.1404,
 0.1597,
 0.1692,
 0.1763,
 0.1805,
 0.1839,
 0.1865,
 0.1879,
 0.1911,
 0.1919,
 0.1921,
 0.1937,
 0.1941,
 0.1953,
 0.1951,
 0.1963,
 0.1961,
 0.1972,
 0.1964,
 0.1971,
 0.1976,
 0.1969,
 0.1972,
 0.1984,
 0.1989,
 0.1988,
 0.2003,
 0.2015,
 0.2028,
 0.2028,
 0.2027,
 0.2032,
 0.2041,
 0.2047,
 0.2048,
 0.206,
 0.2056,
 0.2068,
 0.2068,
 0.207,
 0.2077,
 0.2079,
 0.2083,
 0.2091,
 0.2099,
 0.2098,
 0.2108,
 0.2095,
 0.2098,
 0.2107,
 0.2111,
 0.2114,
 0.2112,
 0.2109,
 0.2112,
 0.2119,
 0.2116,
 0.2109,
 0.2105,
 0.2096,
 0.2102,
 0.2101,
 0.2104,
 0.2099,
 0.2092,
 0.2099,
 0.2096,
 0.2101,
 0.2102,
 0.2098,
 0.2094,
 0.2098,
 0.2095,
 0.2094,
 0.2097,
 0.2096,
 0.21,
 0.2098,
 0.2101,
 0.2107,
 0.2102,
 0.2105,
 0.2104,
 0.2111,
 0.211,
 0.2106,
 0.2107,
 0.2108,
 0.2111,
 0.2112,
 0.2117,
 0.2118,
 0.2122,
 0.2122,
 0.2125,
 0.2121,
 0.2119,
 0.2118,
 0.2117,
 0.2118,
 0.2118,
 0.2119,
 0.2119,
 0.2123,
 0.2127,
 0.2127,
 0.2132,
 0.2132,
 0.2134,
 0.2135,
 0.2136,
 0.213

In [7]:
cf_train

[2.295161446975457,
 2.2798852019021614,
 2.2700846499275076,
 2.2626583119064914,
 2.2575425333437322,
 2.2540343919475134,
 2.2514812968542035,
 2.2499435493560824,
 2.2482103357434617,
 2.2493006251222183,
 2.249626573534049,
 2.250836901596127,
 2.251723681198505,
 2.254671001271081,
 2.256802848911924,
 2.261386917703268,
 2.263888235365379,
 2.268037101232046,
 2.271504289423192,
 2.2755133659767535,
 2.2800290281738747,
 2.28374729809842,
 2.287949239404063,
 2.293038231143196,
 2.2965619769019505,
 2.301065581607602,
 2.3046971512980816,
 2.3101142643942176,
 2.313409760765639,
 2.3185278650687735,
 2.321832024536296,
 2.326487597775877,
 2.3318392901468292,
 2.335997099582045,
 2.340931293073432,
 2.3464020167308774,
 2.3522854150110337,
 2.3568368809042717,
 2.362912664378656,
 2.36880068250207,
 2.3746381025279306,
 2.379504240484329,
 2.385889754230395,
 2.3920941357731573,
 2.398306753205189,
 2.4028394395016788,
 2.4100216668366325,
 2.4151273362465266,
 2.423430953787608

In [48]:
GD_params = f.params(n_batch=10, eta=0.01, n_epochs=200)

In [59]:
def MiniBatchGD(X_, Y_, y_, X_val, Y_val, y_val, GD_params, lambda_=0):

    Npts = X_.shape[0]
    cf_train = []
    cf_val = []
    cl_train = []
    cl_val = []
    acc_train = []
    acc_val = []

    W_1, b_1, W_2, b_2 = f.init_two_layers_w_b_param(X_, Y_, m_=50)

    for epoch in range(GD_params.n_epochs):
        for j in range(Npts // GD_params.n_batch):
            j_start = j * GD_params.n_batch
            j_end = (j + 1) * GD_params.n_batch
            Xbatch = X_[:, j_start:j_end]
            Ybatch = Y_[:, j_start:j_end]

            grad_b_1, grad_b_2, grad_w_1, grad_w_2 = f.ComputeGradsAnalt(
                                                    Xbatch, Ybatch, b_1, b_2, W_1, W_2, lambda_=0)

            W_1 = W_1 - GD_params.eta * grad_w_1
            b_1 = b_1 - GD_params.eta * grad_b_1
            W_2 = W_2 - GD_params.eta * grad_w_2
            b_1 = b_2 - GD_params.eta * grad_b_2

        # cost per epoch
        trainning_cost = f.ComputeCost(X_, Y_, W_1, W_2, b_1, b_2, lambda_=0)
        validation_cost = f.ComputeCost(X_val, Y_val, W_1, W_2, b_1, b_2, lambda_=0)

        # loss per epoch
        trainning_loss = f.ComputeCost(X_, Y_, W_1, W_2, b_1, b_2, 0)
        validation_loss = f.ComputeCost(X_val, Y_val, W_1, W_2, b_1, b_2, 0)

        cf_train.append(trainning_cost)
        cf_val.append(validation_cost)

        cl_train.append(trainning_loss)
        cl_val.append(validation_loss)

        # accuracy per epoch
        acc_train_ = ComputeAccuracy(X_, y_, W_1, W_2, b_1, b_2)
        acc_val_ = ComputeAccuracy(X_val, y_val, W_1, W_2, b_1, b_2)

        acc_train.append(acc_train_)
        acc_val.append(acc_val_)

    return W_1, b_1, W_2, b_2, cf_train, cf_val, cl_train, cl_val, acc_train, acc_val

In [60]:
W_1, b_1, W_2, b_2, cf_train, cf_val, cl_train, cl_val, acc_train, acc_val = MiniBatchGD(
                trainning_data.data[:, 0:100],
                trainning_data.labels[:, 0:100],
                trainning_data.y[0:100],
                validation_data.data[:, 0:100],
                validation_data.labels[:, 0:100],
                validation_data.y[0:100],
                GD_params,
                lambda_=0)

ValueError: operands could not be broadcast together with shapes (50,10) (10,10) 